In [1]:
from __future__ import division, print_function
%matplotlib inline
from importlib import reload  # Python 3
import utils; reload(utils)
from utils import *

Using cuDNN version 5105 on context None
Mapped name None to device cuda0: GeForce GTX TITAN X (0000:04:00.0)
Using Theano backend.


In [2]:
path = "data/imdb/"
model_path = path + 'models/'
if not os.path.exists(model_path): os.mkdir(model_path)

## Setup data

We're going to look at the IMDB dataset, which contains movie reviews from IMDB, along with their sentiment. Keras comes with some helpers for this dataset.

In [3]:
from keras.datasets import imdb
idx = imdb.get_word_index()

This is the word list:

In [4]:
idx_arr = sorted(idx, key=idx.get)
idx_arr[:10]

['the', 'and', 'a', 'of', 'to', 'is', 'br', 'in', 'it', 'i']

...and this is the mapping from id to word

In [5]:
idx2word = {v: k for k, v in idx.items()}

We download the reviews using code copied from keras.datasets:

In [6]:
path = get_file('imdb_full.pkl',
                origin='https://s3.amazonaws.com/text-datasets/imdb_full.pkl',
                md5_hash='d091312047c43cf9e4e38fef92437263')
f = open(path, 'rb')
(x_train, labels_train), (x_test, labels_test) = pickle.load(f)

In [7]:
len(x_train)

25000

Here's the 1st review. As you see, the words have been replaced by ids. The ids can be looked up in idx2word.

In [8]:
', '.join(map(str, x_train[0]))

'23022, 309, 6, 3, 1069, 209, 9, 2175, 30, 1, 169, 55, 14, 46, 82, 5869, 41, 393, 110, 138, 14, 5359, 58, 4477, 150, 8, 1, 5032, 5948, 482, 69, 5, 261, 12, 23022, 73935, 2003, 6, 73, 2436, 5, 632, 71, 6, 5359, 1, 25279, 5, 2004, 10471, 1, 5941, 1534, 34, 67, 64, 205, 140, 65, 1232, 63526, 21145, 1, 49265, 4, 1, 223, 901, 29, 3024, 69, 4, 1, 5863, 10, 694, 2, 65, 1534, 51, 10, 216, 1, 387, 8, 60, 3, 1472, 3724, 802, 5, 3521, 177, 1, 393, 10, 1238, 14030, 30, 309, 3, 353, 344, 2989, 143, 130, 5, 7804, 28, 4, 126, 5359, 1472, 2375, 5, 23022, 309, 10, 532, 12, 108, 1470, 4, 58, 556, 101, 12, 23022, 309, 6, 227, 4187, 48, 3, 2237, 12, 9, 215'

The first word of the first review is 23022. Let's see what that is.

In [9]:
idx2word[23022]

'bromwell'

Here's the whole review, mapped from ids to words.

In [10]:
' '.join([idx2word[o] for o in x_train[0]])

"bromwell high is a cartoon comedy it ran at the same time as some other programs about school life such as teachers my 35 years in the teaching profession lead me to believe that bromwell high's satire is much closer to reality than is teachers the scramble to survive financially the insightful students who can see right through their pathetic teachers' pomp the pettiness of the whole situation all remind me of the schools i knew and their students when i saw the episode in which a student repeatedly tried to burn down the school i immediately recalled at high a classic line inspector i'm here to sack one of your teachers student welcome to bromwell high i expect that many adults of my age think that bromwell high is far fetched what a pity that it isn't"

The labels are 1 for positive, 0 for negative.

In [11]:
labels_train[:10]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

Reduce vocab size by setting rare words to max index.

In [12]:
vocab_size = 5000

trn = [np.array([i if i<vocab_size-1 else vocab_size-1 for i in s]) for s in x_train]
test = [np.array([i if i<vocab_size-1 else vocab_size-1 for i in s]) for s in x_test]
# check the "max index" word out of curiosity
idx2word[vocab_size-1]

'bergman'

Look at distribution of lengths of sentences.

In [13]:
lens = np.array([*map(len, trn)])
(lens.max(), lens.min(), lens.mean())

(2493, 10, 237.71364)

Pad (with zero) or truncate each sentence to make consistent length.

In [14]:
seq_len = 500

trn = sequence.pad_sequences(trn, maxlen=seq_len, value=0)
test = sequence.pad_sequences(test, maxlen=seq_len, value=0)

This results in nice rectangular matrices that can be passed to ML algorithms. Reviews shorter than 500 words are pre-padded with zeros, those greater are truncated.

In [15]:
trn.shape

(25000, 500)

In [16]:
batch_size = 64

## Create simple models

### Single hidden layer NN

The simplest model that tends to give reasonable results is a single hidden layer net. So let's try that. Note that we can't expect to get any useful results by feeding word ids directly into a neural net - so instead we use an embedding to replace them with a vector of 32 (initially random) floats for each word in the vocab.

In [17]:
model = Sequential([
    Embedding(vocab_size, 32, input_length=seq_len),
    Flatten(),
    Dense(100, activation='relu'),
    Dropout(0.7),
    Dense(1, activation='sigmoid')])

In [18]:
model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
flatten_1 (Flatten)          (None, 16000)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               1600100   
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 1,760,201
Trainable params: 1,760,201
Non-trainable params: 0
_________________________________________________________________


In [19]:
model.fit(trn, labels_train, validation_data=(test, labels_test), epochs=2, batch_size=batch_size)

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 1s - loss: 0.4527 - acc: 0.7596 - val_loss: 0.3009 - val_acc: 0.8707
Epoch 2/2
25000/25000 [==============================] - 1s - loss: 0.1968 - acc: 0.9264 - val_loss: 0.3098 - val_acc: 0.8710


The [stanford paper](http://ai.stanford.edu/~amaas/papers/wvSent_acl2011.pdf) that this dataset is from cites a state of the art accuracy (without unlabelled data) of 0.883. So we're short of that, but on the right track.

### Single conv layer with max pooling

A CNN is likely to work better, since it's designed to take advantage of ordered data. We'll need to use a 1D CNN, since a sequence of words is 1D.

In [20]:
conv1 = Sequential([
    Embedding(vocab_size, 32, input_length=seq_len),
    SpatialDropout1D(0.2),
    Dropout(0.2),
    Conv1D(64, 5, padding='same', activation='relu'),
    Dropout(0.2),
    MaxPooling1D(),
    Flatten(),
    Dense(100, activation='relu'),
    Dropout(0.7),
    Dense(1, activation='sigmoid')])

In [21]:
conv1.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [22]:
conv1.fit(trn, labels_train, validation_data=(test, labels_test), epochs=4, batch_size=batch_size)

Train on 25000 samples, validate on 25000 samples
Epoch 1/4
25000/25000 [==============================] - 3s - loss: 0.4597 - acc: 0.7564 - val_loss: 0.2804 - val_acc: 0.8861
Epoch 2/4
25000/25000 [==============================] - 3s - loss: 0.2545 - acc: 0.9051 - val_loss: 0.2689 - val_acc: 0.8890
Epoch 3/4
25000/25000 [==============================] - 3s - loss: 0.2095 - acc: 0.9221 - val_loss: 0.2684 - val_acc: 0.8872
Epoch 4/4
25000/25000 [==============================] - 3s - loss: 0.1791 - acc: 0.9346 - val_loss: 0.2802 - val_acc: 0.8885


That's well past the Stanford paper's accuracy - another win for CNNs!

In [23]:
conv1.save_weights(model_path + 'conv1.h5')

In [24]:
conv1.load_weights(model_path + 'conv1.h5')

## Pre-trained vectors

You may want to look at wordvectors.ipynb before moving on.

In this section, we replicate the previous CNN, but using pre-trained embeddings.

In [27]:
def load_vectors(loc):
    return (load_array(loc+'.dat'),
        pickle.load(open(loc+'_words.pkl','rb')),
        pickle.load(open(loc+'_idx.pkl','rb')))

In [30]:
vecs, words, wordidx = load_vectors('data/glove/results/6B.50d')

The glove word ids and imdb word ids use different indexes. So we create a simple function that creates an embedding matrix using the indexes from imdb, and the embeddings from glove (where they exist).

In [31]:
def create_emb():
    n_fact = vecs.shape[1]
    emb = np.zeros((vocab_size, n_fact))

    for i in range(1,len(emb)):
        word = idx2word[i]
        if word and re.match(r"^[a-zA-Z0-9\-]*$", word):
            src_idx = wordidx[word]
            emb[i] = vecs[src_idx]
        else:
            # If we can't find the word in glove, randomly initialize
            emb[i] = normal(scale=0.6, size=(n_fact,))

    # This is our "rare word" id - we want to randomly initialize
    emb[-1] = normal(scale=0.6, size=(n_fact,))
    emb/=3
    return emb

In [32]:
emb = create_emb()

We pass our embedding matrix to the Embedding constructor, and set it to non-trainable.

In [33]:
model = Sequential([
    Embedding(vocab_size, 50, input_length=seq_len,
              weights=[emb], trainable=False),
    SpatialDropout1D(0.2),
    Dropout(0.25),
    Convolution1D(64, 5, padding='same', activation='relu'),
    Dropout(0.25),
    MaxPooling1D(),
    Flatten(),
    Dense(100, activation='relu'),
    Dropout(0.7),
    Dense(1, activation='sigmoid')])

In [34]:
model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [35]:
model.fit(trn, labels_train, validation_data=(test, labels_test), epochs=2, batch_size=batch_size)

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 2s - loss: 0.6321 - acc: 0.6367 - val_loss: 0.5640 - val_acc: 0.7386
Epoch 2/2
25000/25000 [==============================] - 2s - loss: 0.5518 - acc: 0.7269 - val_loss: 0.4890 - val_acc: 0.7840


We already have beaten our previous model! But let's fine-tune the embedding weights - especially since the words we couldn't find in glove just have random embeddings.

In [36]:
model.layers[0].trainable=True

In [37]:
model.optimizer.lr=1e-4

In [38]:
model.fit(trn, labels_train, validation_data=(test, labels_test), epochs=1, batch_size=batch_size)

Train on 25000 samples, validate on 25000 samples
Epoch 1/1
25000/25000 [==============================] - 2s - loss: 0.5030 - acc: 0.7602 - val_loss: 0.4658 - val_acc: 0.7859


As expected, that's given us a nice little boost. :)

In [39]:
model.save_weights(model_path+'glove50.h5')

## Multi-size CNN

This is an implementation of a multi-size CNN as shown in Ben Bowles' [excellent blog post](https://quid.com/feed/how-quid-uses-deep-learning-with-small-data).

In [40]:
from keras.layers import Merge

We use the functional API to create multiple conv layers of different sizes, and then concatenate them.

In [41]:
graph_in = Input ((vocab_size, 50))
convs = [ ] 
for fsz in range (3, 6): 
    x = Conv1D(64, fsz, padding='same', activation="relu")(graph_in)
    x = MaxPooling1D()(x) 
    x = Flatten()(x) 
    convs.append(x)
out = Concatenate(axis=-1)(convs) 
graph = Model(graph_in, out) 

In [42]:
emb = create_emb()

We then replace the conv/max-pool layer in our original CNN with the concatenated conv layers.

In [43]:
model = Sequential ([
    Embedding(vocab_size, 50, input_length=seq_len, weights=[emb]),
    SpatialDropout1D(0.2),
    Dropout (0.2),
    graph,
    Dropout (0.5),
    Dense (100, activation="relu"),
    Dropout (0.7),
    Dense (1, activation='sigmoid')
    ])

In [44]:
model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [45]:
model.fit(trn, labels_train, validation_data=(test, labels_test), epochs=2, batch_size=batch_size)

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 8s - loss: 0.4818 - acc: 0.7502 - val_loss: 0.2745 - val_acc: 0.8848
Epoch 2/2
25000/25000 [==============================] - 8s - loss: 0.2876 - acc: 0.8850 - val_loss: 0.2578 - val_acc: 0.8946


Interestingly, I found that in this case I got best results when I started the embedding layer as being trainable, and then set it to non-trainable after a couple of epochs. I have no idea why!

In [46]:
model.layers[0].trainable=False

In [47]:
model.optimizer.lr=1e-5

In [48]:
model.fit(trn, labels_train, validation_data=(test, labels_test), epochs=2, batch_size=batch_size)

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 8s - loss: 0.2397 - acc: 0.9050 - val_loss: 0.2469 - val_acc: 0.8984
Epoch 2/2
25000/25000 [==============================] - 8s - loss: 0.2140 - acc: 0.9156 - val_loss: 0.2509 - val_acc: 0.8968


This more complex architecture has given us another boost in accuracy.

## LSTM

We haven't covered this bit yet!

In [49]:
model = Sequential([
    Embedding(vocab_size, 32, input_length=seq_len, mask_zero=True,
              embeddings_regularizer=l2(1e-6)),
    SpatialDropout1D(0.2),
    LSTM(100, implementation=2),
    Dense(1, activation='sigmoid')])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
spatial_dropout1d_4 (Spatial (None, 500, 32)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________


In [50]:
model.fit(trn, labels_train, validation_data=(test, labels_test), epochs=5, batch_size=batch_size)

Train on 25000 samples, validate on 25000 samples
Epoch 1/5
25000/25000 [==============================] - 86s - loss: 0.4603 - acc: 0.7752 - val_loss: 0.3363 - val_acc: 0.8610
Epoch 2/5
25000/25000 [==============================] - 86s - loss: 0.2911 - acc: 0.8843 - val_loss: 0.3022 - val_acc: 0.8775
Epoch 3/5
25000/25000 [==============================] - 88s - loss: 0.2622 - acc: 0.8966 - val_loss: 0.3455 - val_acc: 0.8557
Epoch 4/5
25000/25000 [==============================] - 86s - loss: 0.2305 - acc: 0.9122 - val_loss: 0.3049 - val_acc: 0.8791
Epoch 5/5
25000/25000 [==============================] - 86s - loss: 0.2154 - acc: 0.9184 - val_loss: 0.3231 - val_acc: 0.8675
